In [59]:
import os
import sys
import itertools

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

In [61]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [62]:
MAIN_DIR=os.getcwd()
DATA_DIR=os.path.join(MAIN_DIR,'data')
R_DATA_DIR=os.path.join(DATA_DIR,'raw_data')
N_DATA_DIR=os.path.join(DATA_DIR,'clean_data')

print(MAIN_DIR)
print(DATA_DIR)
print(R_DATA_DIR)
print(N_DATA_DIR)

/mnt/c/workspace/gametheory_oil_market
/mnt/c/workspace/gametheory_oil_market/data
/mnt/c/workspace/gametheory_oil_market/data/raw_data
/mnt/c/workspace/gametheory_oil_market/data/clean_data


In [74]:
crude_price_m = pd.read_csv(os.path.join(N_DATA_DIR,'crude_price_m.csv'), index_col=0)
g20_cli_m = pd.read_csv(os.path.join(N_DATA_DIR,'g20_cli_m.csv'), index_col=0)
crude_price_y = pd.read_csv(os.path.join(N_DATA_DIR,'crude_price_y.csv'), index_col=0)
crude_production_y = pd.read_csv(os.path.join(N_DATA_DIR,'crude_production_y.csv'), index_col=0)
g20_cli_y = pd.read_csv(os.path.join(N_DATA_DIR,'g20_cli_y.csv'), index_col=0)

In [75]:
def plot_order_acf(data):

    f=plt.figure(figsize=(10,15))
    
    ax1_1=f.add_subplot(421)
    ax1_1.set_title('zero order differencing')
    ax1_1.plot(data.values)

    ax1_2=f.add_subplot(422)
    plot_acf(data, ax=ax1_2)
    

    ax2_1=f.add_subplot(423)
    ax2_1.set_title('1st order differencing')
    ax2_1.plot(data.diff().values)

    ax2_2=f.add_subplot(424)
    plot_pacf(data.diff().dropna(), ax=ax2_2)


    ax3_1=f.add_subplot(425)
    ax3_1.set_title('2nd order differencing')
    ax3_1.plot(data.diff().diff().values)

    ax3_2=f.add_subplot(426)
    plot_pacf(data.diff().diff().dropna(), ax=ax3_2)

    ax4_1=f.add_subplot(427)
    ax4_1.set_title('3rd order differencing')
    ax4_1.plot(data.diff().diff().diff().values)

    ax4_2=f.add_subplot(428)
    plot_pacf(data.diff().diff().diff().dropna(), ax=ax4_2)
    plt.show()

In [76]:
plot_order_acf(crude_price_m)

In [77]:
def calc_p_value_adf(data, threshold=0.05):
    # threshold=0.05

    result=adfuller(data)
    _0_result=result[1]
    print('p-value 0 order diff', result[1])

    result=adfuller(data.diff().dropna())
    _1_result=result[1]
    print('p-value 1 order diff', result[1])

    result=adfuller(data.diff().diff().dropna())
    _2_result=result[1]
    print('p-value 2 order diff', result[1])

    result=adfuller(data.diff().diff().diff().dropna())
    _3_result=result[1]
    print('p-value 3 order diff', result[1])

    if _0_result < threshold:
        print("0 order satisfy the criteria")
    elif _1_result < threshold:
        print("1 order satisfy the criteria")
    elif _2_result < threshold:
        print("2 order satisfy the criteria")
    elif _3_result < threshold:
        print("3 order satisfy the criteria")
    else:
        print("0 ~ 3 order does not satisfy ACF condition")

In [78]:
calc_p_value_adf(crude_price_m)

p-value 0 order diff 0.1882124291740272
p-value 1 order diff 1.0811777341588914e-15
p-value 2 order diff 8.715255027258941e-17
p-value 3 order diff 2.4303851702502524e-18
1 order satisfy the criteria


In [79]:
def AIC_PDQS(df):

    '''
    Runs grid search to return lowest AIC result for permutations of pdq/s values in range 0,2
    df - Dataframe to analyze for best pdq/s permutation
    '''

    # Define the p, d and q parameters to take any value between 0 and 2
    p = d = q = range(0, 2)

    # Auto-Regressive (p) -> Number of autoregressive terms.
    # Integrated (d) -> Number of nonseasonal differences needed for stationarity.
    # Moving Average (q) -> Number of lagged forecast errors in the prediction equation.

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    # Run a grid with pdq and seasonal pdq parameters calculated above and get #the best AIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            print(comb, combs)
            try:
                mod = sm.tsa.statespace.SARIMAX(df,order=comb,seasonal_order=combs,
                enforce_stationarity=False,enforce_invertibility=False)

                output = mod.fit()
                ans.append([comb, combs, output.aic])
                print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
            except:
                print('exception')
                continue


    # Find the parameters with minimal AIC value
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
    print(ans_df.loc[ans_df['aic'].idxmin()])
    return ans_df

In [80]:
AIC_PDQS(crude_price_m)

(0, 0, 0) (0, 0, 0, 12)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.48780D+00    |proj g|=  3.79552D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      1      3      1     0     0   1.963D-08   5.488D+00
  F =   5.4877989465930064     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
ARIMA (0, 0, 0) x (0, 0, 0, 12)12 : AIC Calculated =4458.092744633521
(0, 0, 0) (0, 0, 1, 12)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 


At iterate    5    f=  5.07168D+00    |proj g|=  9.83685D-03

At iterate   10    f=  5.06127D+00    |proj g|=  2.15611D-01


/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
 This problem is unconst


At iterate   15    f=  4.89853D+00    |proj g|=  7.38915D-03

At iterate   20    f=  4.89761D+00    |proj g|=  1.60032D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2     20     22      1     0     0   1.600D-06   4.898D+00
  F =   4.8976098930794230     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
ARIMA (0, 0, 0) x (0, 0, 1, 12)12 : AIC Calculated =3980.859233180491
(0, 0, 0) (0, 1, 0, 12)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.25009D+00   

/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
 This problem is unconst

...

/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/mnt/c/workspace/gametheory_oil_market/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.84957D+00    |proj g|=  1.34648D-01


 This problem is unconstrained.



At iterate    5    f=  2.79765D+00    |proj g|=  3.44751D-01

At iterate   10    f=  2.73923D+00    |proj g|=  8.88362D-02

At iterate   15    f=  2.73463D+00    |proj g|=  4.62775D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     19     29      1     0     0   4.140D-06   2.735D+00
  F =   2.7346298999734491     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
ARIMA (1, 1, 1) x (1, 1, 1, 12)12 : AIC Calculated =2230.5194787784408
pdq         (1, 1, 1)
pdqs    (0, 1, 1, 12)
aic       2228.519513
Name: 59, dtype: object


pdq           pdqs          aic
0   (0, 0, 0)  (0, 0, 0, 12)  4458.092745
1   (0, 0, 0)  (0, 0, 1, 12)  3980.859233
2   (0, 0, 0)  (0, 1, 0, 12)  3452.864663
3   (0, 0, 0)  (0, 1, 1, 12)  3346.303701
4   (0, 0, 0)  (1, 0, 0, 12)  3459.942109
..        ...            ...          ...
59  (1, 1, 1)  (0, 1, 1, 12)  2228.519513
60  (1, 1, 1)  (1, 0, 0, 12)  2264.493404
61  (1, 1, 1)  (1, 0, 1, 12)  2261.397834
62  (1, 1, 1)  (1, 1, 0, 12)  2378.149202
63  (1, 1, 1)  (1, 1, 1, 12)  2230.519479

[64 rows x 3 columns]

### ARIMA (1, 1, 1) x (1, 1, 1, 12) : 
### pdq         (1, 1, 1) / pdqs    (0, 1, 1, 12)
### aic       2228.519513 x 2230.519479

## Decided to use ARIMA (1,1,1)

Check whether CLI index leads crude oil price

In [86]:
crude_price_y

Price
Date             
1965-12-31   2.86
1966-12-31   2.88
1967-12-31   2.92
1968-12-31   2.94
1969-12-31   3.09
1970-12-31   3.18
1971-12-31   3.39
1972-12-31   3.39
1973-12-31   3.89
1974-12-31   6.87
1975-12-31   7.67
1976-12-31   8.19
1977-12-31   8.57
1978-12-31   9.00
1979-12-31  12.64
1980-12-31  21.59
1981-12-31  31.77
1982-12-31  28.52
1983-12-31  26.19
1984-12-31  25.88
1985-12-31  24.09
1986-12-31  12.51
1987-12-31  15.40
1988-12-31  12.58
1989-12-31  15.86
1990-12-31  20.03
1991-12-31  16.54
1992-12-31  15.99
1993-12-31  14.25
1994-12-31  13.19
1995-12-31  14.62
1996-12-31  18.46
1997-12-31  17.23
1998-12-31  10.87
1999-12-31  15.56
2000-12-31  26.72
2001-12-31  21.84
2002-12-31  22.51
2003-12-31  27.56
2004-12-31  36.77
2005-12-31  50.28
2006-12-31  59.69
2007-12-31  66.52
2008-12-31  94.04
2009-12-31  56.35
2010-12-31  74.71
2011-12-31  95.73
2012-12-31  94.52
2013-12-31  95.99
2014-12-31  87.39
2015-12-31  44.39
2016-12-31  38.29
2017-12-31  48.05
2018-12-31  61.40
2019-12-31  55.59
2020-12-31  36.86
2021-12-31  65.84
2022-12-31  93.97

In [87]:
g20_cli_y

value
date                 
1965-12-31  101.04040
1966-12-31   99.20785
1967-12-31  100.70470
1968-12-31  101.98770
1969-12-31  100.43950
1970-12-31   99.28596
1971-12-31  100.97950
1972-12-31  104.51430
1973-12-31  101.76050
1974-12-31   96.24492
1975-12-31  100.65930
1976-12-31  101.17110
1977-12-31  101.12930
1978-12-31  102.56300
1979-12-31  100.77380
1980-12-31  100.16700
1981-12-31   99.13705
1982-12-31   98.27892
1983-12-31  101.25350
1984-12-31  100.27240
1985-12-31   99.85706
1986-12-31   99.96777
1987-12-31  100.93570
1988-12-31  101.02380
1989-12-31  100.47050
1990-12-31   98.73935
1991-12-31   99.23318
1992-12-31   98.96948
1993-12-31   99.76413
1994-12-31  100.82910
1995-12-31   99.94540
1996-12-31  100.70430
1997-12-31   99.99880
1998-12-31   98.90424
1999-12-31  101.43300
2000-12-31   99.83041
2001-12-31   98.64306
2002-12-31   98.79036
2003-12-31  100.43760
2004-12-31  100.44390
2005-12-31  100.58950
2006-12-31  101.66120
2007-12-31  101.80510
2008-12-31   95.56057
2009-12-31  100.38240
2010-12-31  100.97180
2011-12-31   99.75735
2012-12-31   99.82371
2013-12-31  100.39900
2014-12-31  100.21470
2015-12-31   99.46811
2016-12-31  100.05780
2017-12-31  100.74670
2018-12-31   99.53818
2019-12-31   99.07269
2020-12-31  100.57450
2021-12-31  100.55260
2022-12-31   98.89095
2023-12-31  100.36840

In [83]:
len(g20_cli_y.values.tolist())


59

In [85]:
len(crude_price_y.values.tolist())


58

In [72]:
crude_cli_spot = pd.DataFrame({
    "crude_price" : crude_price_y.values.tolist(),
    "cli_index" : g20_cli_y.values.tolist()
})
crude_cli_spot

ValueError: All arrays must be of the same length